In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver

## chrome실행
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(5)
driver.get("https://sports.koreanpc.kr/front/club/listClub.do;jsessionid=0E47FE540D3B9FC0C51A3B4D4B356314")

In [2]:
##등록년도 조회
def lookup(year):
    input_text = driver.find_element_by_id('searchYear')
    input_text.clear()
    input_text.send_keys(year)
    driver.find_element_by_class_name("btn_v6").click()
lookup(2020)

In [3]:
##마지막 페이지 찾기
def find_last_page():
    page_bar = driver.find_elements_by_class_name("paging_area")[0]
    pages = page_bar.find_elements_by_tag_name('a')[-1].click()
    page_bar = driver.find_elements_by_class_name("paging_area")[0]
    last_page = page_bar.find_elements_by_tag_name('a')[-3].text
    return int(last_page)

In [4]:
## 마지막 페이지 저장 후 처음으로 이동(초기 셋팅)
last_page = find_last_page()
page_bar = driver.find_elements_by_class_name("paging_area")[0]
page_bar.find_elements_by_tag_name('a')[0].click()

In [5]:
#반복하여 페이지 이동

def next_page():
    BARCOUNT = 2
    current_page = int(driver.find_element_by_class_name("current").text)
    if current_page >= last_page:
        return False
    page_index = current_page%10+BARCOUNT
    page_bar = driver.find_elements_by_class_name("paging_area")[0]
    page_bar.find_elements_by_tag_name('a')[page_index].click()
    if current_page % 10 == 0:    ## 모든 함수 수행 후 실행해야 함
        page_bar = driver.find_elements_by_class_name("paging_area")[0]
        page_bar.find_elements_by_tag_name('a')[-2].click()

In [6]:
##초기 변수 선언
dic = {}
columns = ['번호','지역','클럽명','활동시간','종목','기타종목','장애유형','승인일']
for col in columns:
    dic[col] = list()

In [7]:
#dic에 데이터 저장
while(True):
    if next_page() == False:
        break
    html = driver.page_source
    result = bs(html, 'lxml')#==html.parser
    
    result1 = result.find_all('td',{'data-cell-header':columns})
    result2 = list(map(lambda x : x.text.strip(),result1))
    count = 0
    for i in range(len(result2)):
        if i > 0 and i % 8 == 0:
            count -= 8
        dic[columns[count]].append(result2[i]) 
        count+=1

In [8]:
dic['센터명'] = ['생활체육정보센터']*len(dic['번호'])
fir_df = pd.DataFrame(dic,columns=dic.keys(), index=dic['번호'])
fir_df = fir_df.sort_values('번호', ascending = True)
fin_df= fir_df.drop('번호',axis=1)

In [9]:
fin_df

,지역,클럽명,활동시간,종목,기타종목,장애유형,승인일,센터명
1,경기 양평군,양평군장애인탁구클럽,"매주 수,금 오후1시~5시",탁구,,지체장애,2020-02-06,생활체육정보센터
10,울산 남구,돌고래론볼,일요일오후2시,론볼,,지체장애,2020-01-03,생활체육정보센터
100,서울 강서구,강서뇌성마비복지관축구단,매주 토요일 10시~12시,축구,,뇌병변장애,2020-11-18,생활체육정보센터
1000,부산 북구,부산데프컬링클럽,"매주 수,금 오후22시~24시",컬링,,청각장애,2020-10-21,생활체육정보센터
1001,경남 김해시,신가야 파크골프,"매주 수, 오후 1시~5시",파크골프,,지체장애,2020-09-21,생활체육정보센터
...,...,...,...,...,...,...,...,...
995,울산 울주군,마중물주간보호센터,"매주 월 오전 10시 30분, 수 오후 2시",기타,생활체조,지적장애,2020-09-01,생활체육정보센터
996,경남 김해시,김해장애인탁구동호회 탁사랑,"매주 화,수 오후2시",탁구,,지체장애,2020-09-21,생활체육정보센터
997,경남 김해시,느티나무,"매주 월, 화, 수, 목, 금 오후 1시~5시",게이트볼,,지체장애,2020-09-21,생활체육정보센터
998,경남 김해시,파크골프김해,4시간,파크골프,골프,지체장애,2020-08-21,생활체육정보센터
